In [1]:
print("Setup")
print("https://python.langchain.com/docs/tutorials/llm_chain/#setup")

Setup
https://python.langchain.com/docs/tutorials/llm_chain/#setup


In [2]:
%pip install langchain python-dotenv
%pip install -qU langchain-openai
%pip install langserve[all]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: langserve[all]
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
print("Using Language Models")
print("https://python.langchain.com/docs/tutorials/llm_chain/#using-language-models")

Using Language Models
https://python.langchain.com/docs/tutorials/llm_chain/#using-language-models


In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c4de0ea5-aaa9-4d56-a673-dba507ce1813-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [7]:
print("OutputParsers")
print("https://python.langchain.com/docs/tutorials/llm_chain/#outputparsers")

OutputParsers
https://python.langchain.com/docs/tutorials/llm_chain/#outputparsers


In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)


'ciao!'

In [9]:
chain = model | parser

chain.invoke(messages)

'ciao!'

In [10]:
print("Prompt Templates")
print("https://python.langchain.com/docs/tutorials/llm_chain/#prompt-templates")

Prompt Templates
https://python.langchain.com/docs/tutorials/llm_chain/#prompt-templates


In [11]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [12]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

In [13]:
print("Chaining together components with LCEL")
print("https://python.langchain.com/docs/tutorials/llm_chain/#chaining-together-components-with-lcel")

Chaining together components with LCEL
https://python.langchain.com/docs/tutorials/llm_chain/#chaining-together-components-with-lcel


In [14]:
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})


'ciao'

In [ ]:
print("Serving with LangServe")
print("Client")
print("https://python.langchain.com/docs/tutorials/llm_chain/#client")

In [15]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

ConnectError: [Errno 61] Connection refused